In [1]:
# Librerías estándar
import os
import gzip
import pickle
import zipfile
import json

# Manipulación y análisis de datos
import pandas as pd
import numpy as np

# Scikit-learn
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, normalize
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, precision_score,
    recall_score, f1_score, confusion_matrix
)


# Paso 1.
 Realice la limpieza de los datasets:
 - Renombre la columna "default payment next month" a "default".
 - Remueva la columna "ID".
 - Elimine los registros con informacion no disponible.
 - Para la columna EDUCATION, valores > 4 indican niveles superiores
   de educación, agrupe estos valores en la categoría "others".
 - Renombre la columna "default payment next month" a "default" y remueva la columna "ID".
 - Divida los datasets en x_train, y_train, x_test, y_test.

In [2]:

# 1.1 Cargar los datos
train_data = pd.read_csv("../files/input/train_data.csv.zip", index_col=False, compression="zip")
test_data = pd.read_csv("../files/input/test_data.csv.zip", index_col=False, compression="zip")

# 1.2 Organizar los datasets:

def clean(df):
    df = df.rename(columns={'default payment next month': 'default'})
    df.drop('ID', axis=1, inplace=True)
    df['EDUCATION'] = df['EDUCATION'].apply(lambda x: 4 if x > 4 else x)
    df = df.query('MARRIAGE > 0 and EDUCATION > 0')
    df = df.dropna()
    return df


train_data = clean(train_data)
test_data = clean(test_data)

x_train = train_data.drop(columns=["default"])
y_train = train_data["default"]

x_test = test_data.drop(columns=["default"])
y_test = test_data["default"]



# Paso 3.
 Cree un pipeline para el modelo de clasificación. Este pipeline debe contener las siguientes capas:
- Transforma las variables categoricas usando el método
   one-hot-encoding.
 - Escala las demas variables al intervalo [0, 1].
 - Selecciona las K mejores caracteristicas.
 - Ajusta un modelo de regresion logistica.

In [3]:
# Variables categóricas y numéricas
Categoria = ['SEX', 'EDUCATION', 'MARRIAGE']
numericas = [col for col in x_train.columns if col not in Categoria]

# ColumnTransformer para preprocesar los datos
transformer = ColumnTransformer(
    transformers=[
        ("ohe", OneHotEncoder(), Categoria),
        ('num', StandardScaler(), numericas)
    ],
)

# Pipeline con selección de características y modelo MLP
pipelineMLP = Pipeline(steps=[
    ('transformer', transformer),                  # Preprocesamiento
    ('kbest', SelectKBest(score_func=f_classif)),  # Selección de las K mejores variables
    ('pca', PCA()),                                # PCA con todas las componentes                   # Escalado a [0, 1]
    ('mlp', MLPClassifier(random_state=17,max_iter=15000
                          #early_stopping=True
                          ))
])

# Entrenamiento del modelo
pipelineMLP.fit(x_train, y_train)

print(pipelineMLP.score(x_train,y_train))
print(pipelineMLP.score(x_test,y_test))


0.8217916288836921
0.8281545829156922


# Optimizacion de hiperametros
- Optimice los hiperparametros del pipeline usando validación cruzada.
- Use 10 splits para la validación cruzada.
- Use la función de precision balanceada para medir la precisión del modelo.

In [4]:
param_grid = {
    'pca__n_components': [None],
    'kbest__k':[20],
    "mlp__hidden_layer_sizes": [(50, 30, 40,60)],
    'mlp__alpha': [0.26],
    "mlp__learning_rate_init": [0.001],
}



# Configuración de GridSearchCV
grid_search = GridSearchCV(
    estimator=pipelineMLP,
    param_grid=param_grid,
    cv=10,
    scoring='balanced_accuracy',
    n_jobs=-1,
    refit=True,
    verbose=True
)

# Ajustar el modelo con la búsqueda en cuadrícula
grid_search.fit(x_train, y_train)


Fitting 10 folds for each of 1 candidates, totalling 10 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(transformers=[('ohe',
                                                                         OneHotEncoder(),
                                                                         ['SEX',
                                                                          'EDUCATION',
                                                                          'MARRIAGE']),
                                                                        ('num',
                                                                         StandardScaler(),
                                                                         ['LIMIT_BAL',
                                                                          'AGE',
                                                                          'PAY_0',
                                                                          'PAY_2',
                                                                          'PAY_3',
                                                                          'PAY_4',
                                                                          'PAY_5',
                                                                          'PAY_6',
                                                                          'BILL_AMT1',
                                                                          'BILL_AMT2',
                                                                          'BILL_AMT3',
                                                                          'BILL_AMT4',
                                                                          'BILL_AMT5',
                                                                          'BILL_AMT6',
                                                                          'PAY_AMT1',
                                                                          'PAY_AMT2',
                                                                          'PAY_AMT3',
                                                                          'PAY_AMT4',
                                                                          'PAY_AMT5',
                                                                          'PAY_AMT6'])])),
                                       ('kbest', SelectKBest()), ('pca', PCA()),
                                       ('mlp',
                                        MLPClassifier(max_iter=15000,
                                                      random_state=17))]),
             n_jobs=-1,
             param_grid={'kbest__k': [20], 'mlp__alpha': [0.26],
                         'mlp__hidden_layer_sizes': [(50, 30, 40, 60)],
                         'mlp__learning_rate_init': [0.001],
                         'pca__n_components': [None]},
             scoring='balanced_accuracy', verbose=True)

In [5]:
print(grid_search.best_params_)

{'kbest__k': 20, 'mlp__alpha': 0.26, 'mlp__hidden_layer_sizes': (50, 30, 40, 60), 'mlp__learning_rate_init': 0.001, 'pca__n_components': None}


In [6]:
# Evaluación del mejor modelo
print("Precisión balanceada en entrenamiento:", grid_search.best_score_)
print("Precisión en prueba:", grid_search.score(x_test, y_test))
print("Precisión en prueba:", grid_search.score(x_train, y_train))

Precisión balanceada en entrenamiento: 0.6581299842663441
Precisión en prueba: 0.6695640976303335
Precisión en prueba: 0.6630025835547158


# Guardar el modelo
- Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
- Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

In [7]:
os.makedirs('../files/models', exist_ok=True)

with gzip.open('../files/models/model.pkl.gz', 'wb') as file:
    pickle.dump(grid_search, file)

def cargar_modelo_y_predecir(data, modelo_path="../files/models/model.pkl.gz"):
    try:
        with gzip.open(modelo_path, "rb") as file:
            estimator = pickle.load(file)
        return estimator.predict(data)
    except FileNotFoundError:
        raise FileNotFoundError(f"No se encontró el archivo de modelo en la ruta especificada: {modelo_path}")
    except Exception as e:
        raise RuntimeError(f"Error al cargar el modelo o realizar predicciones: {e}")
    
# Uso de la función
y_train_pred = cargar_modelo_y_predecir(x_train)
y_test_pred = cargar_modelo_y_predecir(x_test)

# Calculo de metricas de precision

- Calcule las metricas de precision, precision balanceada, recall, f1-score para los conjuntos de entrenamiento y prueba.
- Guardelas en el archivo files/output/metrics.json. Cada fila del archivo es un diccionario con las metricas de un modelo.
- Este diccionario tiene un campo para indicar si es el conjunto de entrenamiento o prueba.

# Calculo Matrices de confucion 

- Calcule las matrices de confusion para los conjuntos de entrenamiento y prueba. 
- Guardelas en el archivo files/output/metrics.json. 
- Cada fila del archivo es un diccionario con las metricas de un modelo de entrenamiento o prueba. Por ejemplo:


In [8]:
import os
import json
from sklearn.metrics import accuracy_score, precision_score, balanced_accuracy_score, recall_score, f1_score, confusion_matrix

def metricas(dict_metricas):
    models_dir = '../files/output'
    os.makedirs(models_dir, exist_ok=True)

    if os.path.exists('../files/output/metrics.json'):
        with open('../files/output/metrics.json', mode='r') as file:
            if len(file.readlines()) >= 4:
                os.remove('../files/output/metrics.json')

    with open('../files/output/metrics.json', mode='a') as file:
        file.write(str(dict_metricas).replace("'", '"') + "\n")


def evaluacion(dataset, y_true, y_pred):
    accuracy = float(accuracy_score(y_true, y_pred))
    precision = float(precision_score(y_true, y_pred))
    balanced_accuracy = float(balanced_accuracy_score(y_true, y_pred))
    recall = float(recall_score(y_true, y_pred))
    f1 = float(f1_score(y_true, y_pred))
    metrics = {
        "type": "metrics",
        "dataset": dataset,
        "precision": precision,
        "balanced_accuracy": balanced_accuracy,
        "recall": recall,
        "f1_score": f1
    }

    metricas(metrics)

metrics_train = evaluacion('train', y_train, y_train_pred)
metrics_test = evaluacion('test', y_test, y_test_pred)

def matriz_confusion(dataset, y_true, y_pred):
    matriz = confusion_matrix(y_true, y_pred)
    matrix_confusion = {
        "type": "cm_matrix",
        "dataset": dataset,
        "true_0": {
            "predicted_0": int(matriz[0, 0]),
            "predicted_1": int(matriz[0, 1]),
        },
        "true_1": {
            "predicted_0": int(matriz[1, 0]),
            "predicted_1": int(matriz[1, 1])
        }
    }

    metricas(json.dumps(matrix_confusion))


metrics_train_cm = matriz_confusion('train', y_train, y_train_pred)
metrics_test_cm = matriz_confusion('test', y_test, y_test_pred)


# el mejor que he encontrado
param_grid = {
        "mlp__hidden_layer_sizes": [(3,)],
        "mlp__learning_rate_init": [1],
        "mlp__activation":['logistic'],
        "mlp__learning_rate": ['adaptive'],
        "mlp__max_iter": [2000]
    }


param_grid = {
        "mlp__hidden_layer_sizes": [(3,)],
        "mlp__learning_rate_init": [1],
        "mlp__activation":['logistic'],
        "mlp__learning_rate": ['adaptive'],
        "mlp__max_iter": [2000],
        "mlp__alpha":[0.0001,0.001,0.01,1]
    }
